# Matrix Product States

In [ ]:
using Tenet

There are several Matrix Product States (MPS) realizations in Tenet, each focusing in a different canonical form.

- `MatrixProductState` or `MPS`, which has no canonical form information.
- `MixedCanonicalMatrixProductState` or `MixedCanonicalMPS`, for MPS with a orthogonality center.
- `VidalMatrixProductState` or `VidalMPS`, for MPS in the Vidal gauge.

In this example, we will be using the `MixedCanonicalMPS` which is posibly the one you may want to use the most. In order to construct one, you can pass the arrays.

In [2]:
ψ = MixedCanonicalMPS([rand(ComplexF64, 2,2), rand(ComplexF64, 2, 2, 2), rand(ComplexF64, 2, 2)])

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

`CanonicalForm` or `form` returns the canonical form trait. Note that since we added the arrays directly, and there is no information about the orthogonality center, `MixedCanonicalMPS` interprets it as if the orthogonality center spans all the sites.

In [3]:
form(ψ)

MixedCanonical{Vector{CartesianSite{1}}}(CartesianSite{1}[(1,), (2,), (3,)])

In order to canonize it, you may call the `canonize!` function together with the `CanonicalForm` trait, which in this case should be `MixedCanonical(site_where_to_canonize)`.

In [4]:
canonize!(ψ, MixedCanonical(site"1"))

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

In [5]:
form(ψ)

MixedCanonical{CartesianSite{1}}((1,))

You can directly call `norm` and `normalize!` methods from `LinearAlgebra` too.

In [6]:
norm(ψ)

1.6289505456334052

In [7]:
normalize!(ψ)

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

In [8]:
norm(ψ)

0.9999999999999999

Note that these methods may canonize or move the orthogonality center.

In [9]:
canonize!(ψ, MixedCanonical(all_sites(ψ)))
form(ψ)

MixedCanonical{Vector{CartesianSite{1}}}(CartesianSite{1}[(1,), (2,), (3,)])

In [10]:
norm(ψ)
form(ψ)

MixedCanonical{CartesianSite{1}}((1,))

## Time evolution

### Gate application using the "Simple Update" routine

In order to create an operator, you may use a regular `Tensor` but where the indices are `Index{Plug}`. This way, Tenet knows which `Tensor` indices connect with the `Tensor`s of the MPS.

In [11]:
op = Tensor(rand(ComplexF64, 2, 2, 2, 2), [Index(plug"2"), Index(plug"3"), Index(plug"2'"), Index(plug"3'")])

2×2×2×2 Tensor{ComplexF64, 4, Array{ComplexF64, 4}}:
[:, :, 1, 1] =
 0.604251+0.856644im  0.767446+0.990334im
 0.835433+0.343913im  0.570089+0.605303im

[:, :, 2, 1] =
  0.428553+0.966438im  0.998883+0.0460719im
 0.0873057+0.347208im  0.868642+0.401913im

[:, :, 1, 2] =
  0.190414+0.791398im   0.44739+0.852733im
 0.0497875+0.275271im  0.302613+0.491546im

[:, :, 2, 2] =
 0.162286+0.669582im  0.141545+0.444884im
 0.350267+0.972112im  0.751287+0.856328im

In order to apply the operator, you can call the `simple_update!` function.

In [12]:
simple_update!(ψ, op)

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

As with the `norm` and `normalize!` functions, it canonizes to the acting sites of the gate for numerical precision.

In [13]:
form(ψ)

MixedCanonical{Vector{CartesianSite{1}}}(CartesianSite{1}[(2,), (3,)])

### MPS-MPO contraction

In order to construct a `MatrixProductOperator` or `MPO`, you can directly pass the arrays or construct one random with `rand`.

In [14]:
mpo = rand(MPO; n=3, maxdim=4)

MatrixProductOperator (#tensors=3, #inds=8)

In order to contract the MPS and the MPO, use the general purpose `evolve!` function.

In [15]:
evolve!(ψ, mpo)

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

Note that, currently, the bond dimension of the MPS increases with the evolution.

In [16]:
for bond in all_bonds(ψ)
    println("bond = $bond --> size = $(size(ψ, ind_at(ψ, bond)))")
end

bond = (1,) <=> (2,) --> size = 8
bond = (2,) <=> (3,) --> size = 8


## Compression

In order to reduce the size of the virtual bonds, you may use the `compress!` function. It accepts `maxdim` and `threshold` (which truncates based on `abs`olute value of the singular values) kwargs.

In [17]:
compress!(ψ; maxdim=4)

MixedCanonicalMatrixProductState (#tensors=3, #inds=5)

In [18]:
for bond in all_bonds(ψ)
    println("bond = $bond --> size = $(size(ψ, ind_at(ψ, bond)))")
end

bond = (1,) <=> (2,) --> size = 2
bond = (2,) <=> (3,) --> size = 2
